In [25]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,sum,avg,max

conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
#sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)
csv_df = spark.read.option("header", "True").csv("./web.csv")
csv_df.printSchema()
sc = SparkContext.getOrCreate();


root
 |-- IP: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- Staus: string (nullable = true)



In [50]:
import pyspark.sql.functions as F
# Grouping respones by code
csv_df.groupBy('Staus').count().where(F.length("Staus")==3).show()

+-----+-----+
|Staus|count|
+-----+-----+
|  200|11330|
|  206|   52|
|  302| 3498|
|  404|  251|
|  304|  658|
+-----+-----+



In [70]:
from pyspark.sql.functions import col,sum,avg,max
from pyspark.sql.types import BooleanType
import ipaddress
@F.udf(returnType=BooleanType())
def is_ip(param_ip):
    try:
        ipaddress.ip_address(param_ip)
        return True
    except:
        return False
        
# Grouping by host IP and number of requests.
csv_df.groupBy('IP').count().where((col("count")>10) & (is_ip(col("IP"))==True)).show()

+----------+-----+
|        IP|count|
+----------+-----+
|10.131.2.1| 1626|
|10.128.2.1| 4257|
|10.130.2.1| 4056|
|10.131.0.1| 4198|
|10.129.2.1| 1652|
+----------+-----+



In [106]:
from pyspark.sql.functions import split, explode
@F.udf(returnType=BooleanType())
def is_command(param_comm):
    try:
        if param_comm in ['POST', 'GET', 'HEAD']:
            return True
        else:
            return False
    except:
        return False

# Grouping by command and number of requests.
split_URL = pyspark.sql.functions.split(csv_df['URL'], ' ')
url_df = csv_df.withColumn('URL_COMMAND', split_URL.getItem(0)).groupBy('URL_COMMAND').count().where(is_command(col("URL_COMMAND"))).show(100)

+-----------+-----+
|URL_COMMAND|count|
+-----------+-----+
|       POST|  682|
|       HEAD|    9|
|        GET|15098|
+-----------+-----+



In [144]:
# Grouping by URI and number of requests    

# Drop unnecessary columns
filtered_URL_status = csv_df.drop('IP').drop('Time')

# Select by Status OK
filtered_200 = filtered_URL_status.where("Staus==200")

# Drop URL, adding URI column
URIs = filtered_200.withColumn('URI', split_URL.getItem(1)).drop('URL')

# Group by URI and count
total = URIs.groupBy("URI").count().orderBy(col("count").desc()).show(500)


+--------------------+-----+
|                 URI|count|
+--------------------+-----+
|          /login.php| 3290|
|/js/vendor/modern...| 1345|
|           /home.php|  486|
|/contestproblem.p...|  393|
|  /css/normalize.css|  341|
|/css/bootstrap.mi...|  332|
|/css/font-awesome...|  331|
|      /css/style.css|  328|
|       /css/main.css|  326|
|/bootstrap-3.3.7/...|  315|
|/js/vendor/jquery...|  314|
|        /contest.php|  240|
|        /archive.php|  239|
|/fonts/fontawesom...|  224|
|       /img/ruet.png|  195|
|/standings.php?id=16|  148|
|  /allsubmission.php|  140|
|/bootstrap-3.3.7/...|  140|
|/js/vendor/moment...|  132|
|/login.php?value=...|  128|
|           /sign.php|  127|
|                   /|  121|
|        /compile.php|   96|
|       /compiler.php|   90|
|       /pcompile.php|   80|
|/contestsubmissio...|   79|
|/countdown.php?na...|   71|
| /archive.php?page=2|   55|
|/contestproblem.p...|   49|
|/profile.php?user...|   48|
|/contestsubmissio...|   47|
|  /details.ph